In [ ]:
import trimesh
import numpy as np
import matplotlib.pyplot as plt

from common_functions import *

# Read in geometry defined by Nikolai

In [ ]:
stacked_spheres = trimesh.load_mesh('../sphere-charging/geometry/stacked_spheres.stl') 
stacked_spheres.apply_scale(1e-3)

In [ ]:
stacked_spheres.show()

In [ ]:
## make 2D slices

meshes = stacked_spheres.split(only_watertight=False)
# the convex hull of every component
meshes_convex = [i.convex_hull for i in meshes if i.vertices.shape[0] > 300]

# combine all components into one mesh
convex_combined = np.sum(meshes_convex)

# get a single cross section of the mesh
section = convex_combined.section(plane_origin=stacked_spheres.centroid, plane_normal=[1, 1, 0])
print(convex_combined.centroid)
slice_2D, to_3D = section.to_2D()

# if we want to intersect a line with this 2D polygon, we can use shapely methods
polygons = slice_2D.polygons_full

slice_2D.show()
plt.figure()
for polygon in polygons:
    x, y = polygon.exterior.xy
    plt.plot(x, y, 'k-')
plt.axis('equal')
plt.show()


In [ ]:
import trimesh.exchange.stl

# Get ASCII STL string
ascii_stl_str = trimesh.exchange.stl.export_stl_ascii(mesh=convex_combined)

# Write it to a file
with open("../sphere-charging/geometry/stacked_spheres_edited.stl", "w") as f:
    f.write(ascii_stl_str)

# Define stacked spheres using trimesh

In [ ]:
import numpy as np
import trimesh

# Parameters
radius = 100  # meters (50 microns)
n_rows = 2
n_cols = 4
n_layers = 3

# HCP spacing
dx = 2 * radius                     # Along x
dz = np.sqrt(3) * radius           # Along z (in-plane offset)
dy = 2 * radius #* np.sqrt(2/3)


sphere_template = trimesh.creation.icosphere(subdivisions=2, radius=radius)
spheres = []

for layer in range(n_layers):

    for row in range(n_rows):
        if row % 2 == 0:
            col_start, col_end = 0, n_cols
        else:
            col_start, col_end = 1, n_cols - 1
        for col in range(col_end):
        # for row in range(n_rows ):
        #     print(row%2)
        #     for col in range(n_cols + 1 if row%2 == 0 else 0):
            # Offset x in every other row
            x = col * dx + (radius if row % 2 == 1 else 0)
            z = row * dz
            y = layer * dy
            center = np.array([x, y, z])
            # if 1%(layer+1) == 1:
            #     print(row-layer)
            #     if row-col != 1:
            #         s = sphere_template.copy()
            #         s.apply_translation(center)
            #         spheres.append(s)
            # if 1%(layer+1) == 0:
            #     if row-col != 1:
            #         s = sphere_template.copy()
            #         s.apply_translation(center)
            #         spheres.append(s)

            s = sphere_template.copy()
            s.apply_translation(center)
            spheres.append(s)
hcp_spheres = trimesh.util.concatenate(spheres)
hcp_spheres.apply_translation(-hcp_spheres.centroid)  # center at origin

# get a single cross section of the mesh
hcp_spheres.apply_scale(1e-3)
centroid = hcp_spheres.centroid
# Translate the mesh so that the centroid is at (0, 0, 0)
hcp_spheres.apply_translation(-centroid)

# get a single cross section of the mesh
#hcp_spheres.apply_translation((0.034,0,0))

hcp_spheres.show()

In [ ]:
# Get bounding box corners
min_corner, max_corner = hcp_spheres.bounds  # shape (2, 3)

# Compute size along each axis (X, Y, Z)
size = max_corner - min_corner

print(f"Bounding box min corner: {min_corner}")
print(f"Bounding box max corner: {max_corner}")
print(f"Size (X x Y x Z): {size}")

In [ ]:
# Areas of all triangles (in the mesh surface)
triangle_areas = hcp_spheres.area_faces  # shape: (num_faces,)

# Average area
mean_area = triangle_areas.mean()
print(f"Mean triangle area: {mean_area:.4f}")
# should translate to an area of around 1 micron... in which case we should not go above a map spacing of 2 microns

In [ ]:
centroid = stacked_spheres_cropped.centroid
# Translate the mesh so that the centroid is at (0, 0, 0)
stacked_spheres_cropped.apply_translation(-centroid)
stacked_spheres_cropped.apply_translation((0,0,-0.015))

In [ ]:
hcp_spheres.apply_translation((0,0,-0.015))

In [ ]:
centroid

In [ ]:
import trimesh.exchange.stl

# Get ASCII STL string
ascii_stl_str = trimesh.exchange.stl.export_stl_ascii(mesh=hcp_spheres)

# Write it to a file
with open("../sphere-charging/geometry/stacked_spheres_frompython.stl", "w") as f:
    f.write(ascii_stl_str)


In [ ]:
200* 600* 606.41

In [ ]:
600 *400 * 403.2/(200* 600* 606.41)

In [ ]:
## determine appropiate grid spacing for Electric Field calculations

# Create mesh grid for exact sampling
WorldX, WorldY, WorldZ = 600, 600, 373.20508+30
# 0.6        0.4        0.37320508
stepsize = 10

x_array = np.arange(-WorldX/2, WorldX/2, stepsize)/1000
y_array = np.arange(-WorldY/2, WorldY/2, stepsize)/1000
z_array = np.arange(-WorldZ/2, WorldZ/2, stepsize)/1000

# Create 3D mesh grid
X, Y, Z = np.meshgrid(x_array, y_array, z_array, indexing='ij')

# Flatten the mesh grid to create sampling points
sampling_points = np.column_stack([X.ravel(), Y.ravel(), Z.ravel()])
print(len(sampling_points))
photoelectron_stopping_sites = trimesh.points.PointCloud(sampling_points, colors=[0, 0, 255, 255])

scene = plot_trimesh_edges_only(hcp_spheres, edge_color=[0, 0, 0, 128])
scene.add_geometry([photoelectron_stopping_sites])
scene.show()

In [ ]:
import trimesh.exchange.stl

# Get ASCII STL string
ascii_stl_str = trimesh.exchange.stl.export_stl_ascii(mesh=hcp_spheres)

# Write it to a file
with open("../sphere-charging/geometry/stacked_spheres_frompython_cropped.stl", "w") as f:
    f.write(ascii_stl_str)


In [ ]:
## determine appropiate grid spacing for Electric Field calculations

# Create mesh grid for exact sampling
WorldX, WorldY, WorldZ = 200, 600, 606.61
stepsize = 10

x_array = np.arange(-WorldX/2, WorldX/2, stepsize)/1000
y_array = np.arange(-WorldY/2, WorldY/2, stepsize)/1000
z_array = np.arange(-WorldZ/2, WorldZ/2, stepsize)/1000

# Create 3D mesh grid
X, Y, Z = np.meshgrid(x_array, y_array, z_array, indexing='ij')

# Flatten the mesh grid to create sampling points
sampling_points = np.column_stack([X.ravel(), Y.ravel(), Z.ravel()])
print(len(sampling_points))
photoelectron_stopping_sites = trimesh.points.PointCloud(sampling_points, colors=[0, 0, 255, 255])

scene = plot_trimesh_edges_only(stacked_spheres_cropped, edge_color=[0, 0, 0, 128])
scene.add_geometry([photoelectron_stopping_sites])
scene.show()

In [ ]:
stacked_spheres = trimesh.load_mesh('../sphere-charging/geometry/stacked_spheres_frompython.stl') 

# get a single cross section of the mesh
section = stacked_spheres.section(plane_origin=stacked_spheres.centroid, plane_normal=[0, 1, 0])
slice_2D, to_3D = section.to_2D()

fig, ax = plt.subplots()
ax.axhline(y=0)
slice_2D.show()


In [ ]:
hcp_spheres.centroid